# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6.1

### Форматирование строк

6\.1\.1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [34]:
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv')
new = recipes.sample(n=5)
ids = new['id'].tolist()
minutes = new['minutes'].tolist()

step =  len(str(max(ids)))+4
print(f'|{"id":^{step}}|{"minutes":^{step}}|')
print(f'|{(1+step*2)*"-"}|')
for index in range(len(new)):
    print(f"|{ids[index]:^{step}}|{minutes[index]:^{step}}|")

|    id    | minutes  |
|---------------------|
|  128703  |    25    |
|  159554  |    15    |
|  281745  |    20    |
|  74493   |   105    |
|  313693  |    20    |


6\.1\.2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [40]:
def show_info(name, steps, minutes, author_id):
    steps = "\n".join([f"{i}. {step}" for i, step in enumerate(steps, start=1)])
    res = f'"{name.title()}"\n\n{steps}\n------------------------------\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут'
    return res

print(show_info(name="george s at the cove black bean soup", 
                steps=["clean the leeks and discard the dark green portions",
                       "cut the leeks lengthwise then into one-inch pieces",
                       "melt the butter in a medium skillet , med"], 
                minutes=90, author_id=35193))

"George S At The Cove Black Bean Soup"

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
------------------------------
Автор: 35193
Среднее время приготовления: 90 минут


## Работа с регулярными выражениями

6\.1\.3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [52]:
import re
recipes = pd.read_csv('recipes_sample.csv')
# print(recipes[recipes['id'] == 25082])
df = recipes[recipes['id'] == 25082]['description']
pattern = r'(\d)+\s(hour|hours|minute|minutes)'

for step in df:
    if re.findall(pattern, step):
        print(re.findall(pattern, step))

                         name     id  minutes  contributor_id   submitted  \
2415  basic whole wheat bread  25082      290           25483  2002-04-13   

      n_steps                                        description  \
2415     18.0  a simple, easy whole wheat bread recipe posted...   

      n_ingredients  
2415            NaN  


6\.1\.4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [57]:
pattern = re.compile(r'[^this]\w, but')
recipes = pd.read_csv('recipes_sample.csv')
df = recipes['description'].dropna()
k = 3
for d in df:
    if re.search(pattern, d) and k:
        print(d, '\n')
        k-=1

an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself! 

ok, there are different version of burek (some eastern europeans even greeks) roll a fresh dough and then spend more time rolling, folding and waiting. but if you're like me and my mom, use phyllo! it's quick and you can always keep it in the freezer.  i remember while i was growing up and my mom would make this every sunday! the smell alone coming from our kitchen used to make me come running home. please excuse the directions, but the recipe doesn't really have measurements or written directions! this is an old recipe and no one ever wrote it down. 

tender, garlicky and full of oregano 'n stuff! start this one a day ahead; the beef slices need an overnight

## Лабораторная 6.2

6\.2\.1\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [65]:

df = recipes[recipes['id'] == 72367]['description']
for step in df:
    print(re.sub(r' / ', '/', step))

such a beautiful tart - just like from a european bakery, and delicious to boot! from worldwiderecipes i believe.


### Сегментация текста

6\.2\.2\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [75]:
from nltk.tokenize import word_tokenize
recipes = pd.read_csv('recipes_sample.csv')
words = set()
for steps in recipes['description']:
    if isinstance(steps, str):
        for i in word_tokenize(steps):
            if i.isalpha():
                words.add(i.lower())
print(words,len(words))

{'perrenial', 'arthritis', 'tramp', 'envelopes', 'freezeable', 'billion', 'ventilated', 'struggling', 'catalog', 'dorie', 'soooooooo', 'counterparts', 'staffordshire', 'milk', 'overwhelms', 'aproximate', 'season', 'smoke', 'detect', 'tader', 'euskadi', 'gazpacho', 'london', 'fearful', 'disease', 'fanourios', 'ninjas', 'schlager', 'berggarten', 'coca', 'pepperidge', 'comming', 'mcilhenny', 'survival', 'hock', 'bash', 'plus', 'mizithra', 'savior', 'daal', 'sign', 'bolognese', 'ladyfingers', 'macadamias', 'aggie', 'fantastically', 'competitive', 'basing', 'counted', 'proceeds', 'boyardee', 'creators', 'strainer', 'llama', 'scan', 'glögg', 'diverticulitis', 'boun', 'folk', 'newsgroup', 'pasties', 'ummibrahim', 'kickin', 'ridged', 'couric', 'rom', 'portugese', 'geschnetzeltes', 'engrossed', 'standards', 'pretzles', 'exhibit', 'papery', 'borscht', 'rejoices', 'montery', 'marinating', 'faceed', 'cupcakes', 'finely', 'gardening', 'fascinating', 'serranos', 'cantaloupes', 'maundy', 'ellison', '

6\.2\.3\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [85]:
from nltk.tokenize import 

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,steps_sentenced,sent_num
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,[this wonderful icing is used for icing cakes ...,76
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,[a translucent golden-brown crust allows the g...,27
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,[this is one of the best soups i've ever made ...,24
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,[the first time i made this cake i grated a mi...,23
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,[i wrote this because there are an astounding ...,23


6\.2\.4\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [82]:
from nltk import pos_tag
from nltk import pos_tag, word_tokenize
s1 = 'I  omitted the raspberries and added strawberries instead'
s2 = recipes[recipes['id'] == 241106]['name'].values[0]

def pos_word(word):
    for i in pos_tag(word_tokenize(word)):
        step = len(i[0])+len(i[1])
        print(f'{i[1]: ^{step}}', end = '')
    print('')
    for i in pos_tag(word_tokenize(word)):
        print(f'{i[0]: ^{step}}', end = '')
    print('\n')

pos_word(s1)
pos_word(s2)

PRP    VBD     DT       NNS       CC    VBD         NNS         RB    
    I     omitted    the   raspberries   and     added  strawberries instead 

    JJ       NNS     IN      NNS       VBP     JJ    CC    JJ      NNS   
eggplant  steaks    with   chickpeas  feta    cheese     and     black   olives  

